In [1]:
# data process
import numpy as np
import pandas as pd
import os
import time
import os
import json
from tqdm import tqdm
from multiprocessing import Pool

from process import reduce_memory

have not glove


# 读取数据并规范化

In [2]:
item_feat = pd.read_csv("../data/underexpose_train/underexpose_item_feat.csv", header=None)
item_feat.columns = ["item_id"] + ["vec_" + str(i) for i in item_feat.columns[1:]]
print("缺失情况：", item_feat.isna().sum().sum())
item_feat["vec_1"] = item_feat["vec_1"].map(lambda x:x[1:]).astype("float64")
item_feat["vec_129"] = item_feat["vec_129"].map(lambda x:x[1:]).astype("float64")
item_feat["vec_128"] = item_feat["vec_128"].map(lambda x:x[:-1]).astype("float64")
item_feat["vec_256"] = item_feat["vec_256"].map(lambda x:x[:-1]).astype("float64")
item_feat.head(2)

缺失情况： 0


,item_id,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255,vec_256
0,42844,4.514945,-2.383720,0.500414,0.407068,-1.995229,0.109078,-0.691775,2.227460,-6.437974,...,-3.374727,-1.506969,-1.820180,-3.024644,0.445263,0.013933,-1.300239,2.759948,2.056171,0.508703
1,67898,-2.002905,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,...,-0.538330,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.937197


# 减小内存占用

In [3]:
item_feat = reduce_memory(item_feat, float_=True, int_=False)
item_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108916 entries, 0 to 108915
Columns: 257 entries, item_id to vec_256
dtypes: float32(256), int64(1)
memory usage: 107.2 MB


In [4]:
item_feat.head(2)

,item_id,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255,vec_256
0,42844,4.514945,-2.383720,0.500414,0.407068,-1.995229,0.109078,-0.691775,2.227460,-6.437974,...,-3.374727,-1.506969,-1.820180,-3.024644,0.445263,0.013933,-1.300239,2.759948,2.056171,0.508703
1,67898,-2.002905,-0.929881,0.790017,-1.380895,-0.510463,-1.810096,1.363962,0.497401,-4.038903,...,-0.538330,-2.620164,1.277195,0.601015,-0.345312,0.993457,1.351633,2.162675,2.768597,-0.937197


# 构建txt与img数据向量字典

In [5]:
item_txt_vec_dict = dict(zip(item_feat.iloc[:, 0].values.astype(str), item_feat.iloc[:, 1:129].values))
item_img_vec_dict = dict(zip(item_feat.iloc[:, 0].values.astype(str), item_feat.iloc[:, 129: ].values))

# 多进程余弦相似度计算

In [6]:
def cosine_similarity(item_vec_dict):
    pid = os.getpid()
    print('Run task (%s)...' % (pid))
    start = time.time()
    
    cosine_similarity_dict = {}
    for item_i in tqdm(item_vec_dict):
        for item_j in item_vec_dict:
            if (item_i!=item_j) & (item_i not in cosine_similarity_dict) & (item_i not in cosine_similarity_dict):
                ij_cos = np.sum(item_vec_dict[item_i] * item_vec_dict[item_j])/(np.linalg.norm(item_vec_dict[item_i], ord=2) * np.linalg.norm(item_vec_dict[item_j], ord=2))
                ij_cos = round(ij_cos, 4)
                cosine_similarity_dict.setdefault(item_i, {})
                cosine_similarity_dict[item_i][item_j] = ij_cos
                cosine_similarity_dict.setdefault(item_j, {})
                cosine_similarity_dict[item_j][item_i] = ij_cos
                
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (pid, (end - start)))  
    return cosine_similarity_dict

if __name__=='__main__':
    pool = Pool(5) # 创建一个5个进程的进程池
    result = []
    for i in [item_txt_vec_dict, item_img_vec_dict]:
        result.append(pool.apply_async(func=cosine_similarity, args=(i,)))
    pool.close()
    pool.join()
    """
    遍历result列表，取出子进程对象，访问get()方法，获取返回值。（此时所有子进程已执行完毕）
    result[0].get()
    """
    print('进程结束')

Run task (6162)...


  0%|          | 133/108916 [00:01<20:29, 88.51it/s]

Run task (6163)...


100%|██████████| 108916/108916 [21:15<00:00, 85.40it/s]


Task 6162 runs 1275.29 seconds.


100%|██████████| 108916/108916 [21:26<00:00, 84.65it/s]


Task 6163 runs 1286.62 seconds.
进程结束


In [7]:
np.save('../data/process/txt_cosine_similarity_dict.npy',result[0].get())
np.save('../data/process/img_cosine_similarity_dict.npy',result[0].get())

# # 读取 
# x = np.load('../data/process/txt_cosine_similarity_dict.npy', allow_pickle=True)